# Thesis 2020-2021: Bag-of-Words (BOW) model 

In this notebook, we will create a Bag-of-Words model.

In [1]:
import pandas as pd
import numpy as np
import math

import matplotlib
import matplotlib.pyplot as plt

In [2]:
import csv
    
df_train = pd.read_csv('data/hateval2019_en_train.csv')
df_dev = pd.read_csv('data/hateval2019_en_dev.csv')

df_train_dev = df_train.append(df_dev, ignore_index=True)
df_train_dev = df_train_dev.drop(['TR', 'AG'], axis=1)
df_train_dev

id                                               text  HS
0       201  Hurray, saving us $$$ in so many ways @potus @...   1
1       202  Why would young fighting age men be the vast m...   1
2       203  @KamalaHarris Illegals Dump their Kids at the ...   1
3       204  NY Times: 'Nearly All White' States Pose 'an A...   0
4       205  Orban in Brussels: European leaders are ignori...   0
...     ...                                                ...  ..
9995  19196       @SamEnvers you unfollowed me? Fuck you pussy   0
9996  19197  @DanReynolds STFU BITCH! AND YOU GO MAKE SOME ...   1
9997  19198  @2beornotbeing Honey, as a fellow white chick,...   0
9998  19199  I hate bitches who talk about niggaz with kids...   1
9999  19200  @AnnCoulter @DonaldJTrumpJr You won the" life ...   1

[10000 rows x 3 columns]

In [5]:
df_test = pd.read_csv('data/hateval2019_en_test.csv')
df_test = df_test.drop(['TR', 'AG'], axis=1)
df_test

id                                               text  HS
0     34243  @local1025 @njdotcom @GovMurphy Oh, I could ha...   0
1     30593  Several of the wild fires in #california and #...   0
2     31427  @JudicialWatch My question is how do you reset...   0
3     31694  #Europe, you've got a problem!   We must hurry...   1
4     31865  This is outrageous! #StopIllegalImmigration  #...   1
...     ...                                                ...  ..
2995  31368  you can never take a L off a real bitch😩 im ho...   1
2996  30104  @Brian_202 likes to call me a cunt & a bitch b...   1
2997  31912  @kusha1a @Camio_the_wise @shoe0nhead 1. Never ...   1
2998  31000  If i see and know you a hoe why would i hit yo...   1
2999  30870   You be chasing them hoes fuck what a bitch think   1

[3000 rows x 3 columns]

In [3]:
np.bincount(df_train_dev.HS)

array([5790, 4210], dtype=int64)

In [68]:
import re

from pattern.text.en import singularize
from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer()

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# Create a function to clean the tweets
def cleanTxt(text):
    text = text.lower() # Convert everything to lower case
    text = re.sub(r'@[a-zA-Z0-9]+', '', text) # Remove @mentions
    text = re.sub(r'rt[\s]+', '', text) # Remove RT (retweet symbol)
    text = re.sub(r'&amp;', 'and', text) # Replace '&amp;' by 'and'
    text = re.sub(r'https?:\/\/\S+', '', text) # Remove hyper link  
    #text = re.sub(r'\d+', '0', text) # Replace all numbers by a zero
    text = " ".join([singularize(word) for word in tokenizer.tokenize(text) if word not in stop_words]) # Remove stopwords
    #text = " ".join([singularize(word) for word in text])
    text = re.sub(r'[^\w\s#]', ' ', text) # Remove all non-alphanumeric symbols (excluding whitespace and # characters)
    text = re.sub(r'\s+', ' ', text) # Replace multiple whitespaces by a single whitespace
    text = text.strip() # Remove whitespaces at the beginning and at the end
    
    return text

In [8]:
X_train = df_train_dev.text
X_test = df_test.text
y_train = df_train_dev.HS
y_test = df_test.HS

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer() # Convert a collection of text documents to a matrix of token counts
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [23]:
print(vectorizer.get_feature_names()[3000:3010])
#print(vectorizer.vocabulary_)

['bairn', 'bait', 'baiters', 'baiting', 'baizuo', 'bajuvaria', 'baker', 'bakeries', 'bakerpolito', 'baking']


In [35]:
#X_train.toarray() 
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(10000, 26851) (10000,) (3000, 26851) (3000,)


In [36]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression().fit(X_train, y_train)
y_predict = clf.predict(X_test)

C:\Users\Admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [53]:
# Evaluate using Logistic Regression as the classifier

import import_ipynb
import evaluate # here we import the local evaluate.ipynb jupyter notebook

# Create new test dataframe
df_test_bow = df_test.copy()
df_test_bow['HS'] = y_predict

# Create prediction file for the bow_logreg
df_test_bow[['id', 'HS']].to_csv('predictions/bow_logreg.tsv', sep='\t', index=False, header=False)
df_test_bow[['id', 'HS']].to_csv('input/res/en_a.tsv', sep='\t', index=False, header=False)

# Evaluate the result of the bow_logreg
evaluate.write_eval("scores_bow_logreg")

importing Jupyter notebook from evaluate.ipynb
taskA_fscore: 0.4667915154825536
taskA_precision: 0.6339505564857677
taskA_recall: 0.5669950738916256
taskA_accuracy: 0.508


In [55]:
# Normalize the data via StandardScaler

from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X_train.toarray())
X_train_scaled = scaler.transform(X_train.toarray())
X_test_scaled = scaler.transform(X_test.toarray())

clf_scaled = LogisticRegression().fit(X_train_scaled, y_train)
y_scaled_predict = clf_scaled.predict(X_test_scaled)

In [56]:
# Evaluate using Logistic Regression as the classifier (+ normalized data)

# Create new test dataframe
df_test_bow_scaled = df_test.copy()
df_test_bow_scaled['HS'] = y_scaled_predict

# Create prediction file for the bow_logreg_scaled
df_test_bow_scaled[['id', 'HS']].to_csv('predictions/bow_logreg_scaled.tsv', sep='\t', index=False, header=False)
df_test_bow_scaled[['id', 'HS']].to_csv('input/res/en_a.tsv', sep='\t', index=False, header=False)

# Evaluate the result of the bow_logreg_scaled
evaluate.write_eval("scores_bow_logreg_scaled")

taskA_fscore: 0.49316554122230294
taskA_precision: 0.5981703804221695
taskA_recall: 0.56559934318555
taskA_accuracy: 0.5166666666666667


In [62]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]}
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid.fit(X_train_scaled, y_train)
print("Best cross-validation score: ", grid.best_score_)
print("Best parameters: ", grid.best_params_)

Best cross-validation score:  0.7085
Best parameters:  {'C': 0.001}


In [63]:
# Evaluate using (optimized) Logistic Regression as the classifier (+ normalized data)

logreg_optimized = LogisticRegression(C=0.001).fit(X_train_scaled, y_train)
y_predict_logreg_optimized = logreg_optimized.predict(X_test_scaled)

# Create new test dataframe
df_test_bow_logreg_optimized = df_test.copy()
df_test_bow_logreg_optimized['HS'] = y_predict_logreg_optimized

# Create prediction file for the bow_logreg_optimized
df_test_bow_logreg_optimized[['id', 'HS']].to_csv('predictions/bow_logreg_optimized.tsv', sep='\t', index=False, header=False)
df_test_bow_logreg_optimized[['id', 'HS']].to_csv('input/res/en_a.tsv', sep='\t', index=False, header=False)

# Evaluate the result of the bow_logreg_optimized
evaluate.write_eval("scores_bow_logreg_optimized")

taskA_fscore: 0.5052797948486101
taskA_precision: 0.6029353616915009
taskA_recall: 0.5721948549534757
taskA_accuracy: 0.5253333333333333


In [59]:
# Evaluate using Multinomial Naive Bayes as the classifier

from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB().fit(X_train, y_train)
y_predict_nb = nb.predict(X_test)

# Create new test dataframe
df_test_bow_nb = df_test.copy()
df_test_bow_nb['HS'] = y_predict_nb

# Create prediction file for the bow_nb
df_test_bow_nb[['id', 'HS']].to_csv('predictions/bow_nb.tsv', sep='\t', index=False, header=False)
df_test_bow_nb[['id', 'HS']].to_csv('input/res/en_a.tsv', sep='\t', index=False, header=False)

# Evaluate the result of the bow_nb
evaluate.write_eval("scores_bow_nb")

taskA_fscore: 0.4091336406404223
taskA_precision: 0.6231773655059375
taskA_recall: 0.5408593322386426
taskA_accuracy: 0.474


In [61]:
# Evaluate using Random Forest as the classifier

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier().fit(X_train_scaled, y_train)
y_predict_rf = rf.predict(X_test_scaled)

# Create new test dataframe
df_test_bow_rf = df_test.copy()
df_test_bow_rf['HS'] = y_predict_rf

# Create prediction file for the bow_rf
df_test_bow_rf[['id', 'HS']].to_csv('predictions/bow_rf.tsv', sep='\t', index=False, header=False)
df_test_bow_rf[['id', 'HS']].to_csv('input/res/en_a.tsv', sep='\t', index=False, header=False)

# Evaluate the result of the bow_rf
evaluate.write_eval("scores_bow_rf")

taskA_fscore: 0.38394166280223246
taskA_precision: 0.5601326480416603
taskA_recall: 0.5194033935413246
taskA_accuracy: 0.45266666666666666


## Now let's clean the data and try the optimized Logistic Regression classifier again!

In [69]:
df_train_dev['text_cleaned'] = df_train_dev['text'].apply(cleanTxt)
df_test['text_cleaned'] = df_test['text'].apply(cleanTxt)
df_train_dev

id                                               text  HS  \
0       201  Hurray, saving us $$$ in so many ways @potus @...   1   
1       202  Why would young fighting age men be the vast m...   1   
2       203  @KamalaHarris Illegals Dump their Kids at the ...   1   
3       204  NY Times: 'Nearly All White' States Pose 'an A...   0   
4       205  Orban in Brussels: European leaders are ignori...   0   
...     ...                                                ...  ..   
9995  19196       @SamEnvers you unfollowed me? Fuck you pussy   0   
9996  19197  @DanReynolds STFU BITCH! AND YOU GO MAKE SOME ...   1   
9997  19198  @2beornotbeing Honey, as a fellow white chick,...   0   
9998  19199  I hate bitches who talk about niggaz with kids...   1   
9999  19200  @AnnCoulter @DonaldJTrumpJr You won the" life ...   1   

                                           text_cleaned  
0     hurray saving u many way #lockthemup #buildthe...  
1     would young fighting age man vast majority one...  
2     illegal dump kid border like road kill refuse ...  
3     ny time s nearly white s state pose s array pr...  
4     orban brussel european leader ignoring person ...  
...                                                 ...  
9995                              unfollowed fuck pussy  
9996  stfu bitch go make satanic music u illuminatus...  
9997  honey fellow white chick let tell need shut fu...  
9998  hate bitch talk niggaz kid everybody cant find...  
9999  life time recipient hysterical woman long time...  

[10000 rows x 4 columns]

In [73]:
X_train_cleaned = df_train_dev.text_cleaned
X_test_cleaned = df_test.text_cleaned
y_train = df_train_dev.HS # idem as before
y_test = df_test.HS # idem as before

# Create BOW out of data

vectorizer_cleaned = CountVectorizer() 
X_train_cleaned = vectorizer_cleaned.fit_transform(X_train_cleaned)
X_test_cleaned = vectorizer_cleaned.transform(X_test_cleaned)

# Normalize the data via StandardScaler

scaler_cleaned = preprocessing.StandardScaler().fit(X_train_cleaned.toarray())
X_train_cleaned_scaled = scaler_cleaned.transform(X_train_cleaned.toarray())
X_test_cleaned_scaled = scaler_cleaned.transform(X_test_cleaned.toarray())

In [74]:
param_grid = {'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]}
grid_cleaned = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid_cleaned.fit(X_train_cleaned_scaled, y_train)
print("Best cross-validation score: ", grid.best_score_)
print("Best parameters: ", grid.best_params_)

C:\Users\Admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

Best cross-validation score:  0.7085
Best parameters:  {'C': 0.001}


In [75]:
# Evaluate using (optimized) Logistic Regression as the classifier (+ normalized data)

logreg_cleaned_optimized = LogisticRegression(C=0.001).fit(X_train_cleaned_scaled, y_train)
y_predict_logreg_cleaned_optimized = logreg_cleaned_optimized.predict(X_test_cleaned_scaled)

# Create new test dataframe
df_test_bow_logreg_cleaned_optimized = df_test.copy()
df_test_bow_logreg_cleaned_optimized['HS'] = y_predict_logreg_cleaned_optimized

# Create prediction file for the bow_logreg_cleaned_optimized
df_test_bow_logreg_cleaned_optimized[['id', 'HS']].to_csv('predictions/bow_logreg_cleaned_optimized.tsv', sep='\t', index=False, header=False)
df_test_bow_logreg_cleaned_optimized[['id', 'HS']].to_csv('input/res/en_a.tsv', sep='\t', index=False, header=False)

# Evaluate the result of the bow_logreg_cleaned_optimized
evaluate.write_eval("scores_bow_logreg_cleaned_optimized")

taskA_fscore: 0.5323341354746136
taskA_precision: 0.6046138304637714
taskA_recall: 0.5835112205801861
taskA_accuracy: 0.5436666666666666


## Observations on BOW model:
- Using the optimized Logistic Regression classifier (with CLEANED & normalized data) gives us the best F1-score for the Bag-of-Words model!
- Best F1-score for BOW model: 0.5323341354746136 (still not higher than our best score - 0.57864 - using the dictionary model)
- We can see that normalizing the data (i.e., StandardScaler) improves the the result (F1-score)!
- We can see that cleaning the data (i.e., lowercase, removing stopwords, etc.) improves the result (F1-score)!